# AMPSphere v.2022-03

This is a notebook meant to form the set of notebooks used to analyze the data in AMPSphere and write the manuscript:

__AMPSphere: Global survey of prokaryotic antimicrobial peptides shaping microbiomes__

### Check the c_AMP density of species through different environments controlling-quality of AMPs through the coordinates test

We generate sets of c_AMPs by species and sample excluding those peptides that did not pass the coordinates test. Then group them by environment and select those species happening in at least 2 habitats with at least 10 samples in each. The number of non-redundant c_AMPs attributed to a given species per sample in each sample is compared using Mann-Whitney U test and later the p-values are adjusted using Holm-Sidak.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import combinations, chain
from scipy.stats import spearmanr, mannwhitneyu
from statsmodels.stats.multitest import multipletests as mt

In [2]:
# load data
data = pd.read_table('../data_folder/complete_amps_associated_taxonomy.tsv.gz')

metadata = pd.read_table("../data_folder/metadata.tsv.xz")
metadata = metadata.rename({'sample_accession': 'sample'}, axis=1)

quality = pd.read_table('../data_folder/quality_assessment.tsv.xz')

In [3]:
# preparing data
quality.set_index('AMP', inplace=True)
quality = list(quality[quality.Coordinates == 'Passed'].index)

In [4]:
# filter species-specific genes
data = data[data.amp.isin(quality)]
data = data.loc[data.level == 'species', ['sample', 'source']]
data = data.groupby(['sample', 'source']).agg('size')
data = data.reset_index()
data = data.rename({'source': 'name',
                    0: 'amp_genes'}, axis=1)

In [5]:
genenvo = pd.read_table('../data_folder/general_envo_names.tsv.xz')

metadata = metadata.merge(on=['microontology', 'host_scientific_name', 'host_tax_id'],
                          right=genenvo,
                          how='outer')

metadata = metadata[['sample', 'general_envo_name']]

In [6]:
# calculate densities
sps = pd.read_table('../data_folder/bps-per-sample-per-species.tsv.xz',
                    sep='\t',
                    header='infer')

data = data.merge(on=['sample', 'name'], right=sps)

data['amp_density'] = data['amp_genes'] * 1e6 / data['nbps']

# add metadata
data = data.merge(on='sample', right=metadata)
data.general_envo_name = data.general_envo_name.replace('human saliva', 'human mouth')

In [7]:
def tukeyfence(df):
    itdf = pd.DataFrame()
    for record in df.groupby(['name', 'general_envo_name']):
        record = record[1]
        if len(record) > 9:
            q1, q3 = record['amp_density'].quantile([0.25,0.75])
            iqr = q3-q1
            ul, ll = q3+(1.5*iqr), q1-(1.5*iqr)
            record = record[(record.amp_density <= ul) & (record.amp_density >= ll)]
            itdf = pd.concat([itdf, record])
    return itdf

In [8]:
# remove outliers
df = tukeyfence(data)
df['p'] = df.amp_density / 1e6
df['moe'] = np.sqrt(df.p*(1-df.p)/df.nbps) * 1.96 * 1e6
df.drop('p', axis=1, inplace=True)
df['moe_pct'] = df.moe * 100 / df.amp_density
df.to_csv('species_amp_density_per_sample_hq.tsv.gz',
          sep='\t',
          header=True,
          index=None)
df

,sample,name,amp_genes,taxid,nbps,amp_density,general_envo_name,moe,moe_pct
773023,SAMN06264886,0-14-0-20-30-16 sp002779075,2,44747,1128853,1.771710,groundwater,2.455462,138.592806
789120,SAMN06266445,0-14-0-20-30-16 sp002779075,1,44747,767610,1.302745,groundwater,2.553378,195.999872
835246,SAMN06344006,0-14-0-20-30-16 sp002779075,1,44747,613652,1.629588,groundwater,3.193990,195.999840
972177,SAMN07960327,0-14-0-20-30-16 sp002779075,2,44747,1089727,1.835322,groundwater,2.543624,138.592802
972314,SAMN07960328,0-14-0-20-30-16 sp002779075,2,44747,1100407,1.817509,groundwater,2.518937,138.592803
...,...,...,...,...,...,...,...,...,...
1373903,TARA_142_MES_0.22-3_G,Zunongwangia profunda,1,1762,4485880,0.222922,marine,0.436926,195.999978
1374056,TARA_145_MES_0.22-3_G,Zunongwangia profunda,1,1762,4258030,0.234850,marine,0.460307,195.999977
1374405,TARA_148b_MES_0.22-3_G,Zunongwangia profunda,3,1762,3802238,0.789009,marine,0.892847,113.160608
1374468,TARA_149_MES_0.22-3_G,Zunongwangia profunda,1,1762,3801975,0.263021,marine,0.515521,195.999974


In [9]:
# select species present in >=10 samples per habitats
k = df.groupby(['name', 'general_envo_name']).agg('size')
k = k[k >= 10]

# select species present in more than 1 habitat
k = k.reset_index().groupby('name').apply(lambda x: x.general_envo_name.tolist())
k = k[k.apply(lambda x: len(x)) > 1]
k = k.reset_index()

### Densities per species and habitats controlling quality

In [10]:
# test difference in the amp_densitys among samples of
# the same species through different environments
test = []
for _, s, env in k.itertuples():
    combs = combinations(env, 2)
    for i, j in combs:
        n1 = df[(df.name == s) & (df.general_envo_name == i)]
        n2 = df[(df.name == s) & (df.general_envo_name == j)]
        u, p = mannwhitneyu(n1['amp_density'], n2['amp_density'])
        test.append((s, i, len(n1),
                     n1.amp_density.mean(), n1.amp_density.std(),
                     j, len(n2), n2.amp_density.mean(),
                     n2.amp_density.std(), p))

# format results
test = pd.DataFrame(test, columns=['species',
                                   'env1',
                                   'env1_samples',
                                   'amp_density_avg_e1',
                                   'amp_density_std_e1',
                                   'env2',
                                   'env2_samples',
                                   'amp_density_avg_e2',
                                   'amp_density_std_e2',
                                   'p_value'])

test

# adjust p-values
_, test['p_adj'], _, _ = mt(test['p_value'], method='hs')

# export
test.to_csv('species_amp_density_crossenvironment_hq.tsv.gz', sep='\t', header=True, index=None)
test

/home/celio/miniconda3/lib/python3.9/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


,species,env1,env1_samples,amp_density_avg_e1,amp_density_std_e1,env2,env2_samples,amp_density_avg_e2,amp_density_std_e2,p_value,p_adj
0,0-14-0-80-60-11 sp002779455,groundwater,25,0.997724,0.426723,marine,21,0.944880,0.611923,4.402062e-01,1.000000e+00
1,0-14-0-80-60-11 sp002779455,groundwater,25,0.997724,0.426723,soil,132,1.095878,0.785271,7.789809e-01,1.000000e+00
2,0-14-0-80-60-11 sp002779455,groundwater,25,0.997724,0.426723,water associated,42,0.702492,0.341929,7.426659e-03,1.000000e+00
3,0-14-0-80-60-11 sp002779455,marine,21,0.944880,0.611923,soil,132,1.095878,0.785271,5.723084e-01,1.000000e+00
4,0-14-0-80-60-11 sp002779455,marine,21,0.944880,0.611923,water associated,42,0.702492,0.341929,1.682533e-01,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
7151,Zag111 sp004558955,human gut,53,163.041577,91.571776,pig gut,133,3.930803,3.142374,2.100328e-26,1.484512e-22
7152,Zag111 sp004558955,human gut,53,163.041577,91.571776,primate gut,15,21.175229,18.581715,9.540071e-09,5.839307e-05
7153,Zag111 sp004558955,pig gut,133,3.930803,3.142374,primate gut,15,21.175229,18.581715,8.391519e-05,3.071159e-01
7154,Zoogloea ramigera,activated sludge,61,0.660189,0.353442,wastewater,116,0.685994,0.538758,2.765764e-01,1.000000e+00


In [11]:
### Generating statistics

# select environments with at least 100 samples containing AMPs
k = data[['sample', 'general_envo_name']].drop_duplicates()
k = k.general_envo_name.value_counts()
k = k[k>=100].index

# select pairs containing both environments in the list
x = test[(test.env1.isin(k)) & (test.env2.isin(k))]

n1 = len(set(x.species))
print(f'It was tested {n1} species')

n2 = len(set(x.loc[test.p_adj < 0.05, 'species']))
print(f'''Only {round((n2*100/n1), 2)}% ({n2}) of them
presented significant differences in their AMP density
across different environments''')

savg = (x.amp_density_avg_e1.sum() + x.amp_density_avg_e2.sum()) / (2*len(x))

mmax, mmin = [max(x.amp_density_avg_e1.max(), x.amp_density_avg_e2.max()),
              min(x.amp_density_avg_e1.min(), x.amp_density_avg_e2.min())]

print(f'''In average, it was observed {savg:.2} c_AMP genes per assembled Mbp for each species
per sample, ranging between {mmax:.4}-{mmin:.2}.''')

It was tested 2995 species
Only 39.93% (1196) of them
presented significant differences in their AMP density
across different environments
In average, it was observed 2.4e+01 c_AMP genes per assembled Mbp for each species
per sample, ranging between 2.329e+03-0.12.
